In [12]:
import LocalSQLConnection
import pandas as pd


In [13]:
LocalSQLConnection.main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [14]:
conn = LocalSQLConnection.get_db_connection()

2.6.0


### Question 10

In [15]:
query = """
SELECT *
FROM Bookings
LEFT JOIN Facilities
ON Bookings.facid = Facilities.facid
"""
df = pd.read_sql(query, conn)
df.head()

,bookid,facid,memid,starttime,slots,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,3,1,2012-07-03 11:00:00,2,3,Table Tennis,0.0,5.0,320,10
1,1,4,1,2012-07-03 08:00:00,2,4,Massage Room 1,9.9,80.0,4000,3000
2,2,6,0,2012-07-03 18:00:00,2,6,Squash Court,3.5,17.5,5000,80
3,3,7,1,2012-07-03 19:00:00,2,7,Snooker Table,0.0,5.0,450,15
4,4,8,1,2012-07-03 10:00:00,1,8,Pool Table,0.0,5.0,400,15


In [16]:
df['cost'] = df.apply(lambda row: (row.slots * row.guestcost) if row.memid == 0 else (row.slots * row.membercost), axis=1)

In [17]:
df.head()

,bookid,facid,memid,starttime,slots,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,cost
0,0,3,1,2012-07-03 11:00:00,2,3,Table Tennis,0.0,5.0,320,10,0.0
1,1,4,1,2012-07-03 08:00:00,2,4,Massage Room 1,9.9,80.0,4000,3000,19.8
2,2,6,0,2012-07-03 18:00:00,2,6,Squash Court,3.5,17.5,5000,80,35.0
3,3,7,1,2012-07-03 19:00:00,2,7,Snooker Table,0.0,5.0,450,15,0.0
4,4,8,1,2012-07-03 10:00:00,1,8,Pool Table,0.0,5.0,400,15,0.0


In [18]:
[df.groupby('name').cost.sum() < 1000]

[name
 Badminton Court    False
 Massage Room 1     False
 Massage Room 2     False
 Pool Table          True
 Snooker Table       True
 Squash Court       False
 Table Tennis        True
 Tennis Court 1     False
 Tennis Court 2     False
 Name: cost, dtype: bool]

#### Answer
* Pool Table
* Snooker Table
* Table Tennis

### Question 11

In [19]:
query = """
SELECT m1.surname, m1.firstname, m2.surname AS recommendedby_surname, m2.firstname AS recommendedby_firstname
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER BY m1.surname
"""
df = pd.read_sql(query, conn)
df.head()

,surname,firstname,recommendedby_surname,recommendedby_firstname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren


### Question 12

In [22]:
query = """
SELECT name AS facility, surname, firstname
FROM Bookings
LEFT JOIN Facilities
ON Facilities.facid = Bookings.facid
LEFT JOIN Members
ON Members.memid = Bookings.memid
WHERE Bookings.memid <> 0
"""
df = pd.read_sql(query, conn)
df.head()

,facility,surname,firstname
0,Table Tennis,Smith,Darren
1,Massage Room 1,Smith,Darren
2,Snooker Table,Smith,Darren
3,Pool Table,Smith,Darren
4,Pool Table,Smith,Darren


### Question 13

In [29]:
query = """
SELECT name AS facility, COUNT(bookid) AS bookings
FROM Facilities
LEFT JOIN Bookings
ON Facilities.facid = Bookings.facid
WHERE Bookings.memid <> 0
GROUP BY name
"""
df = pd.read_sql(query, conn)
df.head()

,facility,bookings
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
